Import necessary databases: BeautifulSoup, urllib, time, json, datetime, re, os and pandas.

In [1]:
from bs4 import BeautifulSoup, SoupStrainer  # HTML parsing
import urllib.request  # aufrufen von URLs
from time import sleep  # damit legen wir den Scraper schlafen
import json  # lesen und schreiben von JSON-Dateien
from datetime import datetime  # um den Daten Timestamps zu geben
import re  # regular expressions
import os  # Dateipfade erstellen und lesen
import pandas as pd  # Datenanalyse und -manipulation



Check if folders (one to store a json with all the already visited urls to avoid duplicates, one to sotre the actual scraped data) already exist. If not: create them! Same for the json that holds the already visited urls.

In [2]:
folders = [".../data/visited/", ".../data/Golf/"]
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)
        print(path, "erstellt.")
    else:
        print(folder, "existiert bereits")
    
path_to_visited_urls = ".../data/visited/visited_urls.json"
if not os.path.isfile(path_to_visited_urls):
    with open(path_to_visited_urls, "w") as file:
        json.dump([], file)
        


/Users/michaelacker/Desktop/data/visited/ existiert bereits
/Users/michaelacker/Desktop/data/Golf/ existiert bereits


The variable countries holds the countries that the scraper is supposed to scrape through. It can be either a list of countries (commented) or a single one.

In [3]:
#countries = {"Deutschland": "D",
            # "Oesterreich": "A",
             #"Belgien": "B",
             #"Spanien": "E",
             #"Frankreich": "F",
             #"Italien": "I",
             #"Luxemburg": "L",
             #"Niederlande": "NL"}
countries = {"Deutschland": "D"}


In an endless loop, the scraper will visit the search-page on autoscout24 that holds all offerings for Volkswagen Golf, first registered from 2008, sorted descending by age. From the sourcecode of that page it will idetify all urls that lead to a offered car (the ones who contain "/angebote/") and checks for every url if it is already in the "visited-urls"-list. If no, it appends it to the car_URLs list. This process will be repeated for all of the 20 pages that autoscout24 offers on a simple search.

If after the 20th page there are new (not yet visited cars) in the car_URLs list, it will scrape through all of them and extract a remarkable variety of features for every, appends those features to a dictionary (car_dict) and appends that dictionary finally to a overall dictionary (multiple_cars_dict) that holds all scraped cars.

Last but not least, multiple_car_dict is transfered into a pandas dataframe, several less important columns are deleted and the dataframe is saved as a csv for further wrangling and analysis.

The loop will start again untill stopped manually. If no new car offers are available, the loop will reast for 60seconds befor starting over again.

In [4]:
# initiate counter variables
car_counter = 1
cycle_counter = 0

# initiate endless loop
while True:
    with open(path_to_visited_urls) as file:
        visited_urls = json.load(file)
    
    # If more than 100000 have been visited, visited_urls will be cleared
    if len(visited_urls) > 100000:
        visited_urls = []

    multiple_cars_dict = {}

    cycle_counter += 1
    
    # When more then 1 country has been assigned for scraping, this will iterate through all of them
    for country in countries:
        car_URLs = []
        
        # Loop to iterate through page 1-20 and extract all search results
        for page in range(1, 21):
            try:
                url = "https://www.autoscout24.de/lst/volkswagen/golf?fregfrom=2008&sort=age&"\
                      f"desc=1&page={str(page)}&cy=D&atype=C&ustate=N%2CU&fuel=D%2CB&powertype=kw&custtype=D"\
                      "&ocs_listing=include&search_id=2c1t48wb05m"
                only_a_tags = SoupStrainer("a")
                
                # Create soup objekt for further scraping
                soup = BeautifulSoup(urllib.request.urlopen(url).read(), "html.parser", parse_only=only_a_tags)
            # Excepting handling if url is not valid anymore
            except Exception as e:
                print("Übersicht: " + str(e) + " " * 50, end="\r")
                pass
            
            for link in soup.find_all("a"):
                if r"/angebote/" in str(link.get("href")):
                    car_URLs.append(link.get("href"))
                    
            car_URLs_unique = [car for car in list(set(car_URLs)) if car not in visited_urls]
            print(f'Lauf {cycle_counter} | {country} | Seite {page} | {len(car_URLs_unique)} neue URLs', end="\r")
        print("")
        
       
        if len(car_URLs_unique) > 0:
            counter = 1
            
            # Loop to iterate through all search results and extract data
            for URL in car_URLs_unique:
                print(f'Lauf {cycle_counter} | {country} | Auto {car_counter}' + ' ' * 50, end="\r")
                try:
                    # country, date
                    car_counter += 1
                    car_dict = {}
                    car_dict["country"] = country
                    car_dict["date"] = str(datetime.now())
                    car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.de' + URL).read(), "html5lib")

                    for key, value in zip(car.find_all("dt"), car.find_all("dd")):
                        car_dict[key.text.replace("\n", "")] = value.text.replace("\n", "")

                    # Marke, Modell
                    car_dict["Marke"] = car.find_all("a", attrs={'class': 'css-5xfxbi'})[0].text
                    car_dict["Modell"] = car.find_all("a", attrs={'class': 'css-5xfxbi'})[1].text.split(" ")[-1]
                    
                    # Ort, price
                    car_dict["ort"] = car.find("a", attrs={"class": "css-u2icft"}).text
                    price = car.find("span", attrs={"class": "css-1tk7gp4"}).text
                    price = price[2:price.find(",-")]
                    price = price.replace(".", "")
                    car_dict["price"] = price

                    # Versch Merkmale
                    merkmale = ["Laufleistung", "Getriebe", "Erstzulassung", "Kraftstoff", "Leistung", "Haendler/Privat"]

                    i = 0
                    for tag in car.find_all("div", attrs={'class': 'css-16ceglb'}):
                        tag = str(tag)
                        tag = tag[tag.find(">") + 1:tag.rfind("<")]
                        car_dict[merkmale[i]] = tag
                        i += 1



                    # Extracting from a field of non-seperated, vary data
                    merkmale = []
                    werte = []
                    
                    i = 0
                    for tag in car.find_all("dd", attrs={'class': 'css-uvt9sy'}):
                        tag = str(tag)
                        tag = tag[tag.find(">") + 1:tag.rfind("<")]
                        werte.append(tag)

                    for tag in car.find_all("span", attrs={'class': 'css-1cmkyz9 e1n6hwap0'}):
                        tag = str(tag)
                        tag = tag[tag.find(">") + 1:tag.rfind("<")]
                        merkmale.append(tag)

                    for item in merkmale:
                        try:
                            car_dict[item] = werte[merkmale.index(item)]
                        except:
                            pass

                    for key, value in dict(car_dict).items():
                        if len(value) > 50 or len(key) > 50:
                            del car_dict[key]


                    multiple_cars_dict[URL] = car_dict
                    counter += 1
                    visited_urls.append(URL)
                    if counter > 5:
                        break
                except Exception as e:
                    print("Detailseite: " + str(e) + " " * 50)
                    pass
            print("")
        
        # Action if no new offerings are available
        else:
            print("\U0001F634")
            sleep(60)


    if len(multiple_cars_dict) > 0:
        df = pd.DataFrame(multiple_cars_dict).T

        # delete less important columns
        columns_to_remove = ["Angebotsnummer", "Sicherheit", "Überführungskosten", "Zulassungskosten",
                            "Gesamt, einmalig", "Leasinggesamtbetrag", "Vertragsart", "Fahrleistung p.a.",
                            "Leasingfaktor", "Laufzeit", "Monatliche Rate", "Mehr-km Kosten",
                            "Minder-km Vergütung", "Effektiver Jahreszins", "Sollzins geb. p.a.",
                            "Teilkasko", "Vollkasko", "Schlussrate", "Gänge",
                            "Sonderzahlung", "Folgeraten", "Erste Rate",
                            "Nettodarlehensbetrag", "Nettodarlehen", "Bruttodarlehensbetrag",
                            "Sollzinssatz", "Verfügbar ab", "Unnamed: 0","Bearbeitungsgebühren","Verfügbarkeit",
                             "Letzter Zahnriemenwechsel","Unnamed: 0.1","Stromverbrauch2","Anzahlung"
                            ]
        for n in columns_to_remove:
            try:
                df = df.drop(n, 1)
            except:
                pass
        
        # Save as csv
        df.to_csv(".../Golf/" + re.sub("[.,:,-, ]", "_", str(datetime.now())) + "BEFORE_WRANGLING.csv", sep=";", index_label="url")
    else:
        print("Keine Daten")
    with open(".../data/visited/visited_urls.json", "w") as file:
        json.dump(visited_urls, file)






KeyboardInterrupt: 